## Ejemplo 2: Consultas a la base de datos y construcción de `DataFrames`

### 1. Objetivos:
    - Realizar consultas a la base de datos y construir un `DataFrame` para cada tabla.
 
---
    
### 2. Desarrollo:

#### a) Tablas a `DataFrames`

In [26]:
import pandas as pd
import sqlite3

Realizamos la conexión y obtenemos nuestro índice (cursor) para poder realizar operaciones con la base de datos:

In [27]:
bd = "../Datasets/MovieLens/movielens.sqlite3"
conn = sqlite3.connect(bd)
cur = conn.cursor()
cur

Ahora vamos a obtener la lista de todos los registros de la tabla `users` usando la función `execute(-sql-)` y la instrucción SQL:

`SELECT * FROM users`

In [7]:
sql = "SELECT * FROM users"
cur.execute(sql)

Ahora hay que obtener todos los resultados de la consulta con:
    
`mi_cursor.fetchall()`

In [8]:
resultados = cur.fetchall()

In [12]:
resultados[:5]

[(1, 'F', 1, 10, '48067'),
 (2, 'M', 56, 16, '70072'),
 (3, 'M', 25, 15, '55117'),
 (4, 'M', 45, 7, '02460'),
 (5, 'M', 25, 20, '55455')]

Si recuerdas el último ejemplo, `fetchall` nos regresa una `lista` de `tuplas`. Cada `tupla` representa una fila de nuestro conjunto de datos, por ejemplo obtén el primer registro se podría obtener como:

In [13]:
resultados[0]

(1, 'F', 1, 10, '48067')

O podríamos obtener el género del primer usuario con:

In [14]:
resultados[0][1]

'F'

Algo muy util es obtener la lista de los nombres de las columnas de la tabla consultada, para eso se hacer uso del atributo:

`mi_cursor.description`

In [15]:
cur.description

(('user_id', None, None, None, None, None, None),
 ('gender', None, None, None, None, None, None),
 ('age', None, None, None, None, None, None),
 ('occupation', None, None, None, None, None, None),
 ('cp', None, None, None, None, None, None))

y aplicando listas de compresión se puede obtener la lista deseada:

In [16]:
columnas = list(map(lambda t: t[0], cur.description))
columnas

['user_id', 'gender', 'age', 'occupation', 'cp']

Afortunadamente `pandas` puede recibir justamente `listas` de `tuplas` como ingredientes para construir `DataFrames`. Sólo hace falta indicarle el nombre de las columnas. Los nombres de las columnas están especificados en el [archivo Readme.md](../../Datasets/MovieLens/Readme.md) que venía incluido con el dataset:

Así que ahora se crea el dataframe usando la forma:

`pd.DataFrame(-lista de registros-,
    columns=-lista de nombres de columnas-)`

In [17]:
df = pd.DataFrame(resultados, columns=columnas)
df.head()

,user_id,gender,age,occupation,cp
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Otra manera de realizar esta operación de una manera más compacta es utilizando la siguiente función de Pandas:

`pd.read_sql(-consulta sql-, -variable de conexión a la BD-)`

así que vamos a obtener justo el mismo dataframe y lu guardaremos en la viable `df_1`:

In [18]:
df_1 = pd.read_sql("SELECT * FROM users", conn)
df_1.head()

,user_id,gender,age,occupation,cp
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Sería una buena idea convertir la columna `user_id` en índice para no tener información redundante usando:

`dataframe.set_index(-columna-, drop=True)`

y el resultado lo guardamos en la variable `df_2`:

In [19]:
df_2 = df_1.set_index("user_id", drop=True)
df_2.head(3)

,gender,age,occupation,cp
user_id,,,,
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117


¡Listo! Vamos a guardar nuestros `DataFrames` en el archivo `users.csv` para que no haga falta volver a extraerlos de la base de datos. Ya podemos decirle adiós a nuestro fiel MySQL. ¡Chao, bellisimo!

In [20]:
df_2.to_csv("users.csv")

Ahora hagamos lo mismo para la tabla `occupations`, obtengamos todos los registros, examinemos, obtengamos los nombres de las columnas, creemos el dataframe, reasignemos índices en caso de ser necesario y guardemos el resultado en el archivo `occupations.csv`

In [21]:
occupations = pd.read_sql("SELECT * FROM occupations", conn)
occupations.head()

,occupation_id,description
0,0,other or not specified
1,1,academic/educator
2,2,artist
3,3,clerical/admin
4,4,college/grad student


Haaa y no olvides cerrar tu índice y tu conexión:

In [22]:
occupations_1 = occupations.set_index("occupation_id", drop=True)
occupations_1


,description
occupation_id,
0,other or not specified
1,academic/educator
2,artist
3,clerical/admin
4,college/grad student
...,...
16,self-employed
17,technician/engineer
18,tradesman/craftsman


In [23]:
occupations_1.to_csv("occupations.csv")

In [25]:
pd.read_excel("hola.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'hola.xlsx'

---
---

## Reto 2: Convertir tablas en MySQL a `DataFrames` de `pandas`

### 1. Objetivos:
    - Solicitar todos los datos de las tablas que están almacenadas en nuestra base de datos, convertirlos a `DataFrames` y guardarlos.
    
---
    
### 2. Desarrollo:

#### a) Tablas a `DataFrames`

Ya que tenemos nuestra conexión funcionando adecuadamente, vamos a utilizarla para realizar consultas a las base de datos y construir una base de datos local. Tu Reto consiste en los siguientes pasos:

1. Vuelve a establecer la conexión a la base de datos
2. Usando el comando `SELECT * FROM nombre_de_tabla`, realiza consultas a cada una de las 5 tablas que existen en la base de datos.
3. Crea un `DataFrame` por cada tabla que obtuviste. Para asignarle los nombres de las columnas correctamente, revisa el archivo [Readme.md](../../Datasets/MovieLens/Readme.md) donde está contenida toda esa información.
4. Asegúrate de que el índice sea adecuado en cada `DataFrame`. En los casos en los que haya datos redundantes, convierte una de las columnas en índice.
5. Si lo deseas ordena las columnas de la manera en la que mejor te parezca.
6. Guarda tus `DataFrames` en formato .csv para utilizarlos en los siguientes Retos.

> **NOTA IMPORTANTE**: La tabla movies es un poco complicada porque contiene muchos signos distintos. Tanto en la columna de nombre de película como la de género, encontramos signos como `,`, `:`, `.`, `|`. Esto hace un poco complicado el almacenamiento y lectura de este archivo. Si elijes guardar este archivo como un .csv separado por comas (`,`), a la hora de leerlo de regreso, `pandas` puede confundirse y pensar que el título de una película que contiene comas constituye dos columnas. Por esta razón, te recomiendo que la tabla `movies` la guardes agregando un separador poco convencional como `sep='$'`. De esta manera será muchísimo más fácil leer tu archivo de regreso usando ese separador.

In [28]:
movies = pd.read_sql("SELECT * FROM movies", conn)
movies = movies.set_index("movie_id", drop = True)
movies.to_csv("movies.csv")

Compara con tus compañeros y revisa con la experta para que todos estén seguros de que tienen sus `DataFrames` estructurados de la manera correcta y que sus archivos .csv fueron creados exitosamente. Vamos a utilizar estos archivos en los Retos siguientes, así que es muy importante que tus datos estén estructurados adecuadamente.

In [29]:
movies.shape

(3883, 2)

In [30]:
movies.columns

Index(['title', 'genres'], dtype='object')

In [31]:
movies.head()

,title,genres
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


In [33]:
ratings = pd.read_sql("SELECT * FROM ratings", conn)
#ratings
#ratings = ratings.set_index("rating", drop = True)
ratings.to_csv("ratings.csv")

In [34]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000627 entries, 0 to 3000626
Data columns (total 4 columns):
 #   Column     Dtype
---  ------     -----
 0   user_id    int64
 1   movie_id   int64
 2   rating     int64
 3   timestamp  int64
dtypes: int64(4)
memory usage: 91.6 MB


In [37]:
age_ranges = pd.read_sql("SELECT * FROM age_ranges", conn)
age_ranges = age_ranges.set_index("age_id", drop = True)
age_ranges.to_csv("age_ranges.csv")

In [36]:
age_ranges.head()

,age_id,age_range
0,1,Under 18
1,18,18-24
2,25,25-34
3,35,35-44
4,45,45-49
